In [1]:
from torch_geometric.datasets import SNAPDataset, TUDataset

C:\Users\Abhijit\Documents\GitHub\cpsc490\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# OPTION 1: 10 small/medium graphs, ego-facebook is undirected, twitter/google are directed, interpreable embeddings

dataset = SNAPDataset(root="../data/SNAPDataset", name="ego-facebook")
print(dataset)
dataset[0]

SNAP-ego-facebook(10)


Data(x=[347, 1406], edge_index=[2, 5718], circle=[325], circle_batch=[325])

In [3]:
# OPTION 2: 1K+ social network graphs, but no features on nodes

dataset = TUDataset(root="../data/TUDataset", name="IMDB-BINARY")
print(dataset)
dataset[0]

IMDB-BINARY(1000)


Data(edge_index=[2, 146], y=[1], num_nodes=20)